In [243]:
import numpy as np
import random
from tqdm.auto import tqdm
from dataclasses import dataclass
import copy

In [244]:
problem = np.load('../data/problem_0.npz')
x = problem['x']
# x = np.array([[1,1,1], [1,1,1]])

y = problem['y']


# convert normal python array into numpy ndarray
# x = np.array(x)

PROBLEM_SIZE  = np.shape(x)[0]
PROBLEM_SIZE

2

In [245]:
max_coefficient = 10 # adjustable depending on output scale

def compute_coefficient():
    """returns a random coefficient between 1 and maximum coefficient"""
    return np.random.randint(1, max_coefficient)
    



def are_compatible(operator, value, base=0):
    match operator:
        case "/":
            if not isinstance(value,np.ndarray):
                return False if value == 0 else True
            else : # check if all values are non-zero
                return False if (0 in value) else True
            
        case "^":
            if not isinstance(value, np.ndarray) and not isinstance(base, np.ndarray): # (non array) ^ (non array)
                return False if base < 0 and not isinstance(value, int) else True
            elif not isinstance(value, np.ndarray) and isinstance(base, np.ndarray): # array ^ (non array)
                return False if np.any(base < 0) and not isinstance(value, int) else True
            elif isinstance(value, np.ndarray) and not isinstance(base, np.ndarray): # (non array) ^ array
                return False if base < 0 and any((not isinstance(i, int)) for i in value) else True
            else : # array ^ array
                for i in range(len(value)):                    
                    if (base[i] < 0 and not isinstance(value[i], int)):
                        return False 
                return True
            
        case "log" :
            if not isinstance(value,np.ndarray):
                return False if value <= 0 else True
            else : # check if all values are non-negative
                return False if (np.any(value <= 0)) else True
        
        case "arccos" :
            if not isinstance(value,np.ndarray):
                return False if value < -1 or value > 1 else True
            else : # check if all values are between -1 and 1
                return False if (np.any(value < -1 ) or np.any(value > 1) ) else True
            
        case "arcsin" :
            if not isinstance(value,np.ndarray):
                return False if value < -1 or value > 1 else True
            else : # check if all values are between -1 and 1
                return False if (np.any(value < -1 )or np.any(value > 1) )else True
        
        case "sqrt" :
            if not isinstance(value,np.ndarray):
                return False if value < 0 else True
            else : # check if all values are non-negative
                return False if (np.any(value < 0) )else True

        case "reciprocal" :
            if not isinstance(value,np.ndarray):
                return False if value == 0 else True
            else:
                return False if (np.any(value == 0)) else True
            
        case "tan" :
            if not isinstance(value,np.ndarray):
                k = (value - np.pi / 2) / np.pi
                return False if k.is_integer() else True
            else:
                for i in range(len(value)):
                    k = (value[i] - np.pi / 2) / np.pi
                    if k.is_integer():
                        return False
                return True

        case _:
            return True


In [246]:
op = "arcsin"
val = np.array([0.5, 0.4, -0.2])
print(are_compatible(op, val))

a = np.array([1, 2, 3])
b = np.array([4, 5, 6])
print(np.dot(a,b))
print(np.multiply(a,b))


True
32
[ 4 10 18]


In [247]:

BINARY_OPERATORS = {
    "+": np.add,
    "-": np.subtract,
    "*": np.multiply,
    "/": np.divide,
    "^": np.pow
}

#https://numpy.org/doc/2.1/reference/routines.math.html
UNARY_OPERATORS = {
        "": lambda x: x,  
        "sin": np.sin,
        "cos": np.cos,
        "tan":np.tan,
        "log": np.log,
        "arccos": np.arccos,
        "arcsin":np.arcsin,
        "arctan":np.arctan,
        "sqrt":np.sqrt,
        "cbrt":np.cbrt,
        "abs":np.abs,
        "reciprocal":np.reciprocal
    }

#VARIABLES = [f"X_{i}" for i in range(PROBLEM_SIZE)]

#VARIABLES_WEIGHTS = [[1/len(VARIABLES) for _ in range(len(VARIABLES))]]
VARIABLES_MAP = {f"X_{i}": x[i] for i in range(PROBLEM_SIZE)}    # {'X_0': [1, 2, 3], 'X_1': [4, 5, 6], 'X_2': [7, 8, 9]}
LEAVES = [i for i in range(10)] + list(VARIABLES_MAP.keys())
# print(VARIABLES_MAP)
# print(LEAVES)

## Printing functions

In [248]:
def print_tree(node, is_root: bool = True):
    """
    Prints the symbolic representation of the tree with the names of the variables
    
    Args:
        node: TreeNode
        is_root: bool
    """
    if not node:
        return

    # Add parentheses around subexpressions unless it's the root
    if not is_root:
        print("(", end="")

    # Traverse the left child
    if node.left:
        print_tree(node.left, is_root=False)

    # Print the current node's value
    print(f"{node.coefficient} {node.value}" if node.coefficient and node.value in VARIABLES_MAP else node.value, end=" ")

    # Traverse the right child
    if node.right:
        print_tree(node.right, is_root=False)

    # Close parentheses if not the root
    if not is_root:
        print(")", end="") 


def print_tree_values(node, is_root: bool = True):
    """ 
    Prints the symbolic representation of the tree with the values of the variables
    
    Args:
        node: TreeNode
        is_root: bool
    """
    if not node:    
        return

    # Add parentheses around subexpressions unless it's the root
    if not is_root:
        print("(", end="")

    # Traverse the left child
    if node.left:
        print_tree_values(node.left, is_root=False)

    # Print the current node's value
    print(f"{node.coefficient} * {VARIABLES_MAP[node.value]}" if node.coefficient and node.value in VARIABLES_MAP else node.value, end=" ")

    # Traverse the right child
    if node.right:
        print_tree_values(node.right, is_root=False)

    # Close parentheses if not the root
    if not is_root:
        print(")", end="") 


def print_expr(node):
    """
    Prints the symbolic representation of the tree with the names of the variables inside an expression
    """
    print_tree(node) 
    print(" = y")

def print_expr_values(node):
    """
    Prints the symbolic representation of the tree with the values of the variables inside an expression
    """
    print_tree_values(node) 
    print(" = ", end="")
    print(node.evaluate_tree_from_node())

## Tree structure


In [249]:
class TreeNode:
    def __init__(self, value):
        self.value = value      # This can be an operator or operand
        self.left = None        # Left child
        self.right = None       # Right child
        self.coefficient = None # multiplicative coefficient for a variable
    
    def __copy__(self):
        return TreeNode(self.value, self.left, self.right, self.coefficient)

    def __eq__(self, other):
        if not isinstance(other, TreeNode):
            return False
        return self.value == other.value

    def get_nodes_from_node(self):
        """ 
        Returns a list of all nodes in the tree starting from the given node
        """
        nodes = []
        if self:
            nodes.append(self)
        if self.left:
            nodes.extend(self.left.get_nodes_from_node())
        if self.right:
            nodes.extend(self.right.get_nodes_from_node())
        return nodes
    
    def get_non_leaves_nodes_from_node(self):
        """
        Returns a list of all non leaf nodes in the tree starting from the given node
        """
        nodes = []
        if self and (self.left or self.right):
            nodes.append(self)
        if self.left:
            nodes.extend(self.left.get_non_leaves_nodes_from_node())
        if self.right:
            nodes.extend(self.right.get_non_leaves_nodes_from_node())
        return nodes
    
    def get_leaves_nodes_from_node(self):
        """
        Returns a list of all leaf nodes in the tree starting from the given node
        """
        nodes = []
        if self and not (self.left or self.right):
            nodes.append(self)
        if self.left:
            nodes.extend(self.left.get_leaves_nodes_from_node())
        if self.right:
            nodes.extend(self.right.get_leaves_nodes_from_node())
        return nodes
        
    def validate_tree_from_node(self):
        """
        Returns True if the tree is syntactically correct without checking domain constraints of operators, False otherwise
        """
        if not self:
            return True
        
        if self.value in BINARY_OPERATORS:
            if not self.left or not self.right:
                return False  # Operators must have two children
            return self.left.validate_tree_from_node() and self.right.validate_tree_from_node()
        
        elif self.value in UNARY_OPERATORS:  # Allow unary operators
            if self.right and not self.left:
                return self.right.validate_tree_from_node()
            return False  # Unary operators must have one child on the right
        
        # elif self.value in VARIABLES_MAP and isinstance(self.value, str):  # Allow variables
        elif self.value in LEAVES:
            return True
        else:
            return False  # Invalid value
    
# (3 + 2) * (4 + 5)        Treenode (value = *, left = Treenode (value = +, left = 3, right = 2), right = Treenode (value = +, left = 4, right = 5))
    def evaluate_tree_from_node(self):
        """
        Returns the value of the expression represented by the tree starting form a specific node
        """
        if not self:
            raise ValueError("Cannot evaluate an empty tree.")
        
        # Check if it's a binary operator
        if self.value in BINARY_OPERATORS:
            left_val = self.left.evaluate_tree_from_node()
            right_val = self.right.evaluate_tree_from_node()
            return BINARY_OPERATORS[self.value](left_val, right_val)
        
        # Check if it's a unary operator
        elif self.value in UNARY_OPERATORS:
            right_val = self.right.evaluate_tree_from_node() # Typically applies to right child
            return UNARY_OPERATORS[self.value](right_val)  # Correct unary application
        
        # Check if it's a variable
        elif self.value in VARIABLES_MAP:
            # return VARIABLES_MAP[self.value]  # Lookup the variable value
            return np.multiply(self.coefficient, VARIABLES_MAP[self.value])  # Lookup the variable value
        
        # Check if it's a numeric constant or coefficient
        elif isinstance(self.value, (int, float)):
            return self.value  # Return as-is for numeric leaf selfs
        
        # If none of the above, it's an error
        else:
            raise ValueError(f"Invalid self value: {self.value}")

    def print_tree_from_node(self):
        print_expr(self)

    def print_tree_values_from_node(self):
        print_expr_values(self)


class Tree:
    def __init__(self, root, depth):
        self.root = root
        self.depth = depth
    
    def __copy__(self):
        return Tree(self.root, self.depth)
    
    def get_nodes(self):
        return self.root.get_nodes_from_node()

    def get_non_leaves_nodes(self):
        return self.root.get_non_leaves_nodes_from_node()
    
    def get_leaves_nodes(self):
        return self.root.get_leaves_nodes_from_node()
    
    def validate_tree(self):
        return self.root.validate_tree_from_node()
    
    def evaluate_tree(self):
        return self.root.evaluate_tree_from_node()
    
    def print_tree(self):
        self.root.print_tree_from_node()

    def print_tree_values(self):
        self.root.print_tree_values_from_node()


    

def random_initial_tree(depth, maxdepth, variables, binary_operators, unary_operators):
    if depth == maxdepth:  # Add a variable until they are all chosen, if yes add a number
        if variables:
            var = random.choice(variables)
            leaf = TreeNode(var)
            leaf.coefficient = compute_coefficient()
            variables.remove(var)
        else:
            leaf = TreeNode(compute_coefficient())
            leaf.coefficient = 1
        return leaf
    
    elif depth == maxdepth - 1: # Add a unary operator
        node = TreeNode(None)
        node.right = random_initial_tree(depth + 1, maxdepth, variables, binary_operators, unary_operators)
        node.left = None
        if random.choice([0, 1]): # 50% chance of invariant unary operator, 50% chance of any of the other unary operators
            node.value = ""
        else:
            available_unary = [op for op in unary_operators if are_compatible(op, np.multiply(VARIABLES_MAP[node.right.value], node.right.coefficient) if node.right.value in VARIABLES_MAP else int(node.right.value))]
            node.value = random.choice(available_unary) # If a choice of a variant unary operator was made, choose a random variant from all the possible ones
        return node
    
    else: # Add a binary operator
        node = TreeNode(None)
        node.left = random_initial_tree(depth + 1, maxdepth, variables, binary_operators, unary_operators)
        node.right = random_initial_tree(depth + 1, maxdepth, variables, binary_operators, unary_operators)
        available_binary = [op for op in binary_operators if are_compatible(op, node.right.evaluate_tree_from_node(), node.left.evaluate_tree_from_node())]
        node.value = random.choice(available_binary) # Choose a random binary operator from all the possible ones
        return node
    
def get_parent(root, target):
    """
    Find the parent of a target node in the tree.
    """
    if not root or root == target:
        return None
    if root.left == target or root.right == target:
        return root
    return get_parent(root.left, target) or get_parent(root.right, target)

def validate_after_replacement(root, replaced_node, unary_operators, binary_operators):
    """
    Validate the tree after replacing a subtree.

    Args:
        root (TreeNode): The root of the tree.
        replaced_node (TreeNode): The node that was replaced.
        unary_operators (list): List of unary operators.
        binary_operators (list): List of binary operators.

    Returns:
        bool: True if the tree is valid, False otherwise.
    """
    current = replaced_node
    while current:
        parent = get_parent(root, current)
        if (parent and parent.value in unary_operators and not are_compatible(parent.value, current.evaluate_tree_from_node())) \
        or (parent and parent.value in binary_operators and not are_compatible(parent.value, current.evaluate_tree_from_node(), parent.left.evaluate_tree_from_node())):
            return False
        current = parent  # Traverse up to the root
    return True
    
def swap_subtrees(source_tree, target_tree):
    # Tree A, Tree B
    # nodoA = random.choice(lista di nodi di A)
    unary_operators = list(UNARY_OPERATORS.keys())
    binary_operators = list(BINARY_OPERATORS.keys())
    source_nodes = source_tree.get_nodes()
    source_node = random.choice(source_nodes)
    # listB = lista di nodi di B tranne le leaves di B
    target_nodes = target_tree.get_non_leaves_nodes()
    #available_target_nodes = [node for node in target_nodes if node in source_node.get_nodes_from_node()]
    while target_nodes:
        # target_node -> parent del nodo in target_tree che verrà sostituito con source_tree
        target_node = random.choice(target_nodes)
        # target_node è valido solo se tutte le sue leaves sono comprese nelle leaves di source_node
        while not all(leaf in source_node.get_leaves_nodes_from_node() for leaf in target_node.get_leaves_nodes_from_node()):
            target_nodes.remove(target_node)
            if not target_nodes:
                return False
            target_node = random.choice(target_nodes)

        if (target_node.value in unary_operators and are_compatible(target_node.value, source_node.evaluate_tree_from_node())):
            tmp = target_node.right
            target_node.right = source_node
            if validate_after_replacement(target_tree.root, target_node, unary_operators, binary_operators):
                return True
            target_node.right = tmp

        elif (target_node.value in binary_operators):
            choice = random.choice(["right", "left"])
            if choice == "right" and are_compatible(target_node.value, source_node.evaluate_tree_from_node(), target_node.left.evaluate_tree_from_node()):
                tmp = target_node.right
                target_node.right = source_node
                if validate_after_replacement(target_tree.root, target_node, unary_operators, binary_operators):
                    return True
                target_node.right = tmp
            elif choice == "left" and are_compatible(target_node.value, target_node.right.evaluate_tree_from_node(), source_node.evaluate_tree_from_node()):
                tmp = target_node.left
                target_node.left = source_node
                if validate_after_replacement(target_tree.root, target_node, unary_operators, binary_operators):
                    return True
                target_node.left = tmp
        target_nodes.remove(target_node)
    return False

In [250]:
def generate_initial_solution():
    variables = list(VARIABLES_MAP.keys())
    n_variables = len(variables)
    n_leaves = int(2 ** np.ceil(np.log2(n_variables)))
    n_actual_leaves = n_leaves * 2
    binary_operators = list(BINARY_OPERATORS.keys())
    unary_operators = list(UNARY_OPERATORS.keys())
    unary_operators.remove("")
    max_depth = np.log2(n_actual_leaves)

    while True:
        variables = list(VARIABLES_MAP.keys())
        root = random_initial_tree(0, max_depth, variables, binary_operators, unary_operators)
        try:
            root.print_tree_from_node()
            if root.validate_tree_from_node():
                root.evaluate_tree_from_node()
                tree = Tree(root, max_depth)
                return tree
        except:
            pass
            
""" 
def wrapper(x):
    tree = generate_initial_solution(x)
    return evaluate_tree(tree.root)
"""

' \ndef wrapper(x):\n    tree = generate_initial_solution(x)\n    return evaluate_tree(tree.root)\n'

## steps
- We treat a possible solution as a tree. The tree has attribute root, which is the root of the tree of class TreeNode and max_depth.
- Generate random tree
    - we need each variable at least once 
    - each variable has exactly one coefficient chosen as a random float number in the range [?, ?]
    - each variable has exactly one unary operator
    - unary operator is chosen as: 50% chance of "" (i.e. no change to the variable), 50% chance of choosing among all other unary operators
        - check if the unary operator is appliable to the variable ->
            ```
            leaves_map = {}
            for e in leaves:
                available_unary_operators = [op for op in list(UNARY_OPERATORS.keys()) if op.is_applicable(e)]
                chosen_unary_operator = 50% chance of "" (i.e. no change to the variable), 50% chance of choosing among [available_unary_operators]
                leaves_map[e] = [chosen_unary_operator]
            # leaves = [-2, 3]
            # leaves_map = {-2: square, 3: log}
            for e in leaves:
                node = leaves_map[e]
                node.left = null
                node.right = e
                # insert node to tree
            ```
    - number of leaves = nearest power of two greater than keys.length()
    - number of actual leaves = [number of leaves] * 2
    - number of coefficients = [number of leaves] - keys.length()
    - number of binrary operators = total number of nodes in  tree with [number of leaves] leaves - [number of leaves]]
    - validate tree
    - if valid, return tree
    - else, ?
- Example:
    - x.length() = 3
    - number of leaves = 4
    - number of actual leaves = 8
    - number of coefficients = 1
    - number of operands = 3

    ```bash
                    +
            /                  \
            *                    +
        /      \           /        \
      u        1          1          u
    /   \    /   \      /   \       /  \
    nul  *  nul   *    nul    *     nul *
    ```
### EA approach
- Individual is rapresented as a tree and a fitness
    - fitness is a tuple of 2 values: (-mse, right_sign_100)
        - right_sign_100 is the percentage of correct sign predictions
        - mse is the mean squared error
- Classic Genetic Programming
    •Key elements 
    •Representation: tree structures
    •Recombination: exchange of subtrees
    •Mutation: random change in trees
        - subtree mutation -> replace  entire subtree
        - point  mutation -> change single node
        - permutation -> exchange node right with left
        - hoist -> take subtree and make it root
        - expansion -> take random leaf and replace it with a new subtree
        - collapse -> take a subtree and replace it with leaf
        - 
    •Population model: generational
    •Parent selection: fitness proportional
    •Survivor selection: deterministic

In [251]:
random_tree = generate_initial_solution()
# random_tree.print_tree()
random_tree2 = generate_initial_solution()
# random_tree2.print_tree()
if random_tree.validate_tree() and random_tree2.validate_tree():
    print("valid")
    # random_tree.print_tree_values()
    # nodes = random_tree.get_nodes()
    # for node in nodes:
    #     print(node.value)

    # try swap
    cp1 = copy.deepcopy(random_tree)
    cp2 = copy.deepcopy(random_tree2)

    if swap_subtrees(cp1, cp2):
        print("swapped")
        cp2.print_tree()
        cp2.print_tree_values()
    else:
        print("not swapped")

    # source_node = random.choice(source_nodes)
    # listB = lista di nodi di B tranne le leaves di B
    # target_nodes = target_tree.get_non_leaves_nodes()
    # available_target_nodes = [node for node in target_nodes if node in source_node.get_nodes()]
    # target_node = random.choice(available_target_nodes)


( (4 X_0 ))- (sin (5 X_1 )) = y
( (7 X_1 ))/ (arctan (1 X_0 )) = y
valid
swapped
( (sin (5 X_1 )))/ (arctan (1 X_0 )) = y
( (sin (5 * [ 0.8905889   0.13291573 -0.7681345   0.03013576  0.4779096  -0.55487934
  0.60503608 -0.82682564 -0.09702707 -0.92454678 -0.56563361  0.77068825
  0.1642847   0.7174899  -0.75720985  0.67010917  0.23942876 -0.84363105
  0.82259868  0.97802134 -0.86211507  0.83943732 -0.21602585  0.0177975
 -0.65146487 -0.82602941 -0.80516642  0.72467008 -0.62405164 -0.2395128
  0.34125778  0.45492905  0.79628167 -0.73588324  0.73667825 -0.64838493
  0.2005171   0.16917248  0.52559842  0.73126664 -0.98720236  0.41328357
  0.37898247 -0.5557004   0.17949884  0.00806806 -0.93734626  0.94883082
 -0.56311284 -0.31623598  0.27576768 -0.02805208 -0.79436998  0.37901698
  0.33420331 -0.38774321  0.43419835 -0.73518045  0.56533203  0.75662714
 -0.06384705 -0.471218    0.64122427 -0.05604378 -0.81148114  0.30612991
 -0.24766922 -0.40534289 -0.67343045  0.43593555 -0.36705503 -0.0

## EA

In [252]:
@dataclass
class Individual:
    genome: Tree
    fitness: tuple

In [253]:
def mse(y_computed: np.ndarray, y_expected: np.ndarray):
    # 100*np.square(y_train-d3584.f(x_train)).sum()/len(y_train):g}")
    return 100 * np.square(y_expected - y_computed).sum() / len(y_expected)

def fitness(sol: Tree):
    y_computed = sol.root.evaluate_tree_from_node()
    right_sign_100 = 100 * np.sum(np.sign(y_computed) == np.sign(y)) / len(y)
    return  -mse(y_computed, y), right_sign_100

In [254]:
POPULATION_SIZE = 10

MAX_ITERATIONS = 1000

In [255]:
initial_population = [Individual(generate_initial_solution(), 0) for i in range(POPULATION_SIZE)]
# set the fitness
for ind in initial_population:
    ind.fitness = fitness(ind.genome)

print(initial_population)

for i in tqdm(range(MAX_ITERATIONS)):
    # select parents
    parents = random.choices(initial_population, k=2)

    # reproduce
    c1 = copy.deepcopy(parents[0].genome)
    c2 = copy.deepcopy(parents[1].genome)

    # crossover
    swap_subtrees(c1, c2)

    c_fitness = fitness(c2)
    # add to population
    initial_population.append(Individual(c2, c_fitness))

    # sort population
    initial_population.sort(key=lambda x: x.fitness[0], reverse=False)

    # remove worst individual
    initial_population.pop()

initial_population.sort(key=lambda x: x.fitness[0], reverse=False)
print("Best individual has formula:")
initial_population[0].genome.print_tree()
print(f"Best individual has fitness: ({initial_population[0].fitness[0]}, {initial_population[0].fitness[1]})")

# print(initial_population.sort(key=lambda x: x.fitness[0], reverse=False)[0].fitness)
# print(initial_population.sort(key=lambda x: x.fitness[0], reverse=False)[0])

# # y = x+ n
# # mse(evaluate_tree(initial_solution, {}), y[0])
# # tree
# #   operator
# #       |
# #      / \
# #    x    n

# for i in range(200):
#     sol = TreeNode(initial_solution.value)
#     sol.left = initial_solution.left
                                
#     # mutation
#     sol.value = operators[random.randint(0,num_operators)]
#     sol.right =  TreeNode(random.randint(1,10))

#     ev = evaluate_tree(sol, {})
#     # print(ev)
#     if mse(evaluate_tree(initial_solution,{}), y[0]) > mse(ev,y[0]):
#         initial_solution.value = sol.value
#         initial_solution.right = sol.right
#         print("found better solution")
#         print(mse(evaluate_tree(sol, {}),y))


# print(evaluate_tree(initial_solution,{}))
# print(f"{initial_solution.left.value} {initial_solution.value} {initial_solution.right.value}")
    



(cbrt (1 X_1 ))- (cbrt (9 X_0 )) = y
( (6 X_1 ))- ( (6 X_0 )) = y
(reciprocal (9 X_1 ))/ (reciprocal (4 X_0 )) = y
(reciprocal (4 X_0 ))+ ( (3 X_1 )) = y
( (2 X_0 ))- ( (4 X_1 )) = y
( (6 X_1 ))/ ( (4 X_0 )) = y
(sin (5 X_0 ))+ (arccos (1 X_1 )) = y
(abs (4 X_1 ))/ (sin (7 X_0 )) = y
( (7 X_1 ))* (reciprocal (2 X_0 )) = y
( (5 X_1 ))* ( (7 X_0 )) = y
[Individual(genome=<__main__.Tree object at 0x000001F3B190A8D0>, fitness=(np.float64(-1784.2007118803585), np.float64(2.2))), Individual(genome=<__main__.Tree object at 0x000001F3B190B260>, fitness=(np.float64(-17722.903468660163), np.float64(7.8))), Individual(genome=<__main__.Tree object at 0x000001F3B1129C10>, fitness=(np.float64(-85099.9578534655), np.float64(50.2))), Individual(genome=<__main__.Tree object at 0x000001F3B376D130>, fitness=(np.float64(-1437.9893394407773), np.float64(54.3))), Individual(genome=<__main__.Tree object at 0x000001F3B376C1D0>, fitness=(np.float64(-910.2014555502315), np.float64(86.2))), Individual(genome=<__

  0%|          | 0/1000 [00:00<?, ?it/s]

C:\Users\giorg\AppData\Local\Temp\ipykernel_27780\1233658273.py:3: RuntimeWarning: overflow encountered in square
  return 100 * np.square(y_expected - y_computed).sum() / len(y_expected)
C:\Users\giorg\AppData\Local\Temp\ipykernel_27780\1233658273.py:3: RuntimeWarning: overflow encountered in scalar multiply
  return 100 * np.square(y_expected - y_computed).sum() / len(y_expected)


Best individual has formula:
(reciprocal (9 X_1 ))/ (( (reciprocal ((reciprocal (9 X_1 ))/ (( ( (9 X_1 )))* (( ( (5 X_1 )))* ( (reciprocal ((reciprocal (5 X_1 ))/ (( ( (5 X_1 )))* (( ( (5 X_1 )))* (( (( ( ( (reciprocal ((reciprocal (9 X_1 ))/ (( ( (reciprocal ((reciprocal (9 X_1 ))/ (( (9 X_1 ))* (( ( ( (reciprocal ((reciprocal (9 X_1 ))/ (( ( (5 X_1 )))* (( ( ( (reciprocal ((reciprocal (9 X_1 ))/ (( (reciprocal ((reciprocal (9 X_1 ))/ (( (9 X_1 ))* (( ( (5 X_1 )))* ( (reciprocal ((reciprocal (5 X_1 ))/ (( ( ( ( (5 X_1 )))))* (( ( (5 X_1 )))* ( (reciprocal ( (5 X_0 ))))))))))))))* (( ( (5 X_1 )))* ( (reciprocal ((reciprocal (5 X_1 ))/ (( ( ( ( (5 X_1 )))))* (( (5 X_1 ))* ( (reciprocal ( (5 X_0 ))))))))))))))))* ( (reciprocal ((reciprocal (9 X_1 ))/ (( (reciprocal ((reciprocal (9 X_1 ))/ (( (9 X_1 ))* (( ( ( (reciprocal ((reciprocal (9 X_1 ))/ (( ( (5 X_1 )))* (( ( ( (reciprocal ((reciprocal (9 X_1 ))/ (( (9 X_1 ))* (( ( (5 X_1 )))* ( (reciprocal ((reciprocal (5 X_1 ))/ (( ( ( ( (5 X_1 